# Task C 2. model and streaming application

# ID: 27173186 Name: Shengming Zhao
# ID:29593387 Name:Jipeng Yin

In [7]:

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'

import ast
import sys
import time
import json
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import geohash
#client = MongoClient()
#db = client.fit5148_assignment_db
#db.drop_collection('fit5148_collection')
def sendDataToDB(iter):
    
    client = MongoClient()
    db = client.fit5148_assignment_db
    collection = db.fit5148_collection
    record_lst=[[],[],[]]
    
    for record in iter:
        record=ast.literal_eval(record[1]) #recover its original data type and structure
        
        record_lst[record['Sender ID']].append(record)
        print(record_lst[0])# add to coresponding list
        # 0 for producer 1, 1 for producer 2, 2 for producer 3
    
    if len(record_lst[1])>0 and len(record_lst[2])>0: # if 2 data from TERRA and AQUA
            # calculate hash value of 2 locations
            hashvalue1= geohash.encode(record_lst[1][0]['Latitude'],
                                       record_lst[1][0]['Longitude'],5)
            
            hashvalue2=geohash.encode(record_lst[2][0]['Latitude'],
                                       record_lst[2][0]['Longitude'],5)
        
            if (hashvalue1==hashvalue2): #if 2 locations are same
                # calculate average of temperature and confidence
                temperature=(record_lst[2][0]['Surface Temperature (Celcius)']+
                             record_lst[1][0]['Surface Temperature (Celcius)'])/2
                
                confidence=(record_lst[2][0]['Confidence']+record_lst[1][0]['Confidence'])/2
                # assign average temperature and confidence to 2 data.
                record_lst[1][0]['Surface Temperature (Celcius)']=temperature
                record_lst[1][0]['Confidence']=confidence
                record_lst[2][0]['Surface Temperature (Celcius)']=temperature
                record_lst[2][0]['Confidence']=confidence
                
                
                
                #if  receive data from producer 2 
    if len(record_lst[1])>0:
        for i0 in range(len(record_lst[0])):
            #calculate hash value for producer 1, and 2
            hashvalue0= geohash.encode(record_lst[0][i0]['Latitude'],
                                       record_lst[0][i0]['Longitude'],5)
            hashvalue1=geohash.encode(record_lst[1][0]['Latitude'],
                                       record_lst[1][0]['Longitude'],5)
            #if same location, join them
            if (hashvalue0==hashvalue1):
                if 'Hotspot' in record_lst[0][i0].keys():
                    temp=record_lst[0][i0]['Hotspot']
                    temp.append(record_lst[1][0])
                    
                    record_lst[0][i0]['Hotspot']=temp
                else:
                    record_lst[0][i0]['Hotspot']=record_lst[1][0]
                print(record_lst[0][i0])
                
                    
                
         #empty data from producer 2   
        record_lst[1]=[]
         #if  receive data from producer 3 
    if len(record_lst[2])>0:
        for i0 in range(len(record_lst[0])):
            
            hashvalue0= geohash.encode(record_lst[0][i0]['Latitude'],
                                       record_lst[0][i0]['Longitude'],5)
            hashvalue2=geohash.encode(record_lst[2][0]['Latitude'],
                                       record_lst[2][0]['Longitude'],5)
            
            if (hashvalue0==hashvalue2):
                if 'Hotspot' in record_lst[0][i0].keys():
                    temp=record_lst[0][i0]['Hotspot']
                    temp.append(record_lst[2][0])
                    
                    record_lst[0][i0]['Hotspot']=temp
                else:
                    record_lst[0][i0]['Hotspot']=record_lst[2][0]
                print(record_lst[0][i0])
                
                    
                   
           #empty data from producer 3         
        record_lst[2]=[]
                    
    for i in range(len(record_lst[0])):
       #insert climate data to mongo db
        try:
            collection.insert(record_lst[0][i])
            
            
        except Exception as ex:
            print("Exception Occured. Message: {0}".format(str(ex)))
    
        
            
                    
    
    client.close()

n_secs = 10 # seconds of interval
topic = "2717318" #topic name

conf = SparkConf().setAppName("fit5148_taskc").setMaster("local[2]")# 2 threads
sc = SparkContext.getOrCreate()
if sc is None:
    sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, n_secs) 
    
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {
                        'bootstrap.servers':'127.0.0.1:9092', 
                        'group.id':'27173186-group', 
                        'fetch.message.max.bytes':'15728640',
                        'auto.offset.reset':'largest'})
                        # Group ID is completely arbitrary

lines = kafkaStream.foreachRDD(lambda rdd: rdd.foreachPartition(sendDataToDB))

ssc.start()
time.sleep(600) # Run stream for 10 minutes just in case no detection of producer
# ssc.awaitTermination()
ssc.stop(stopSparkContext=True,stopGraceFully=True)
